In [2]:
!pip install flask econml scikit-learn matplotlib


In [2]:
from flask import Flask, request, render_template_string
import pandas as pd
import numpy as np
from econml.dml import LinearDML
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
import matplotlib.pyplot as plt
import io
import base64
import threading

app = Flask(__name__)


In [3]:
!pip install causalml 

In [4]:
def perform_econml_dml(df):
    try:
        #encoding gender column
        df['gender'] = df['gender'].map({'male': 1, 'female': 0})
        X = df[['age', 'gender']]    
        treatment = df['treatment']   
        y = df['outcome']  
        est = LinearDML(model_y=LassoCV(),
                model_t=RandomForestRegressor(),
                discrete_treatment=False,
                random_state=0)
        est.fit(X=X, T=treatment, Y=y)
        ate_effects = est.effect(X)  
        ate_value = np.mean(ate_effects)
         # Plot
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.hlines(y=ate_value, xmin=0, xmax=1, colors="green", linestyles="--", label=f"ATE = {ate_value:.2f}")
        ax.set_title("Average Treatment Effect (ATE)")
        ax.set_xlabel("Samples")
        ax.set_ylabel("Treatment Effect")
        ax.set_ylim(ate_value - 0.5, ate_value + 0.5)
        ax.legend()

        buf = io.BytesIO()
        plt.tight_layout()
        plt.savefig(buf, format="png")
        buf.seek(0)
        image_base64 = base64.b64encode(buf.read()).decode('utf-8')
        buf.close()
        plt.close(fig)

        return ate_value, image_base64
    except Exception as e:
        return f"Error: {str(e)}", None

In [5]:


upload_form = '''
<!doctype html>
<title>EconML DML CSV Upload</title>
<h2>Upload CSV for Double Machine Learning (EconML)</h2>
<form method=post enctype=multipart/form-data>
  <input type=file name=csv_file>
  <input type=submit value=Upload>
</form>
{% if ate or ate_psm %}
<hr>
<h3>Estimated Effects</h3>
<ul>
  {% if ate %}
  <li><strong>ATE (DML):</strong> {{ ate }}</li>
  <img src="data:image/png;base64,{{ plot_data }}" alt="ATE Plot">
  {% endif %}

  {% if ate_psm %}
  <li><strong>ATE (PSM):</strong> {{ ate_psm }}</li>
  {% endif %}
</ul>
{% endif %}

{% if error %}
<p style="color:red;"><strong>{{ error }}</strong></p>
{% endif %}

'''


@app.route('/', methods=['GET', 'POST'])
def upload_and_analyze():
    ate = None
   
    plot_data = None
    error = None

    if request.method == 'POST':
        file = request.files.get('csv_file')
        if file and file.filename.endswith('.csv'):
            try:
                df = pd.read_csv(file)
                
                # Call both backend functions
                ate_result, plot_data = perform_econml_dml(df)
            

                # Error handling
                if isinstance(ate_result, str):
                    error = f"DML Error: {ate_result}"
               
                else:
                    ate = f"{ate_result:.4f}"
                    
            except Exception as e:
                error = f"Processing error: {str(e)}"
        else:
            error = "Please upload a valid CSV file."

    return render_template_string(upload_form, ate=ate, plot_data=plot_data, error=error)


In [3]:
def run_flask():
    app.run(port=5000, debug=True,use_reloader=False)

threading.Thread(target=run_flask).start()


NameError: name 'threading' is not defined